In [ ]:
# Parameters
day_obs = '2024-06-26'
instrument = 'latiss'

In [ ]:
import os
import pandas as pd
import matplotlib.pylab as plt
from IPython.display import display, Markdown, display_markdown

from lsst.summit.utils import ConsDbClient

In [ ]:

URL = "http://consdb-pq.consdb:8080/consdb" # Don't use this one
os.environ["no_proxy"] += ",.consdb"
%matplotlib inline
day_obs_int = int(day_obs.replace('-', ''))



In [ ]:
client = ConsDbClient(URL)
display_markdown('### Consolidated Database is accessible',raw=True)

In [ ]:

psf_query = f'''
    SELECT psf_area FROM cdb_lsstcomcamsim.ccdvisit1_quicklook
    where day_obs = {day_obs_int}
'''

sky_brightness_query = f'''
'''

zero_point_query = f'''
    SELECT zero_point FROM cdb_lsstcomcamsim.ccdvisit1_quicklook
    where day_obs = {day_obs_int}
'''

ra_dec_query = f'''
    SELECT s_ra, s_dec FROM cdb_lsstcomcamsim.ccdvisit1_quicklook
    where day_obs = {day_obs_int}
'''


In [ ]:
day_obs_int = int(day_obs.replace('-', ''))
instrument = 'lsstcomcamsim'
ccdvisit1_quicklook = f''' 
    SELECT * FROM cdb_{instrument}.ccdvisit1_quicklook
'''

visit1 = f'''
    SELECT * FROM cdb_{instrument}.visit1
'''

# Join Visit1 and ccdVisit1 to access data and day obs
visits = client.query(visit1).to_pandas()
quicklook = client.query(ccdvisit1_quicklook).to_pandas()

visits = visits.join(quicklook, on='visit_id',lsuffix='',rsuffix='_q')



In [ ]:
visits_w_psf = visits[visits['psf_area'].notna()]

visits_w_zero = visits[visits['zero_point'].notna()]


In [ ]:
# Pare down to only day obs 
visits_today = visits[(visits['day_obs'] == day_obs_int)]# & (visits['psf_area'].notna())]

ra = visits_today['s_ra']
dec = visits_today['s_dec']

In [ ]:
plt.style.use('seaborn-v0_8-bright')
def plot(y, x):
    # Times Square Tutorial
    #x = np.linspace(0, 10, 100)
    #y = y0 + A * np.sin(2 * np.pi * x / lambd)

    # plot
    fig, ax = plt.subplots()

    ax.scatter(x, y)#, linewidth=2.0, color="#058b8c")

    #ax.set(
    #    xlim=(0, 8), xticks=np.arange(1, 8),
    #    ylim=(0, 8), yticks=np.arange(1, 8)
    #)

    plt.show()

In [ ]:
display_markdown('# This plot is only proof that data exists', raw=True)
plot(ra, dec)

In [ ]:
#try:
#    psf_data = client.query(psf_query).to_pandas()
#except requests.HTTPError or requests.JSONDecodeError:
#    psf_data = client.query(psf_query).to_pandas()
